In [3]:
#30/7/24 Creates a K fold KNN model for each file in a folder provided they are Fingerprints

import os
import pandas as pd
import numpy as np
import math
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sys
sys.path.insert(1, '/Users/james/Documents/Honours/Python')
from Function_Repo import metriccalc 
from Function_Repo import setbalance
import math

seed = 81
#Data import
datasets = []
index = []
directory = '/Users/james/Documents/Honours/Data/Fingerprints/'
namelist = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if 'csv' in filename:
        pathname = directory + file
        df = pd.read_csv(pathname)
        df.drop(columns=['SMILES'], inplace=True)
        df = setbalance(df)
        datasets.append(df)
        namelist.append(filename)

sets = 0
finalresults = []
for df in datasets:
    df = df.dropna()
    dfarray = df.to_numpy()
    
    dfarray, tempset = train_test_split(dfarray, test_size=0.2, 
                                        random_state=seed, stratify = df['Toxicity_Value'])
    
    size = len(dfarray) / 5
    splitsize = math.ceil(size)
    empty = [[] for _ in range(5)]
    splits = []
    for list in empty:
        splits.append(list)
        
    loops = 0
    currsplit = 0
    #Results in a list of 5 lists that each contain 1/5 of the targetdata
    for row in dfarray:
        splits[currsplit].append(row)
        if loops == splitsize:
            loops = 0
            currsplit = currsplit + 1
        loops = loops + 1

    folds = []
    for split in splits:
        temp = pd.DataFrame(split)
        folds.append(temp)
    
    model_list = []
    validlist = []
    totalmetrics = []
    testdat = []
    #For loop that uses each fold once for valid/testing and the rest for training
    #Each 'split' in the range corresponds to the set used for test/validation with the other 4 for training
    iteration = 1
    print('========================================')
    print('For dataset', namelist[sets])
    for split in range(0,5):
        #Set creation
        trainlist = [df for i, df in enumerate(folds) if i != split]
        trainset = pd.concat(trainlist, axis=0)
        testset, validset = train_test_split(folds[split], test_size=0.5, random_state=seed)

        ytrain = trainset[0].values
        xtrain = trainset.iloc[:, 1:]

        ytest = testset[0].values
        xtest = testset.iloc[:, 1:]

        yvalid = validset[0].values
        xvalid = validset.iloc[:, 1:]

        #lists of metric values
        mcclist = []
        bestmetrics = 1
        maxval = 0
        
        length = len(xtrain)
        nneigh = 100
        if length < nneigh:
            nneigh = length
        for mtry in range(1, nneigh):
            #using mtry as the adjusted hyperparameter creates a series of random forests
            rf = KNeighborsClassifier(n_neighbors=mtry, weights='distance', 
                            algorithm='auto', leaf_size=100, p=2, metric='minkowski', 
                            metric_params=None, n_jobs=4)

            model= rf.fit(xtrain, ytrain)
            preds = model.predict(xtest)
            #calculate metric (mcc)
            mcc = metriccalc(preds, ytest)[12]
            mcclist.append(mcc)
            if mcc > maxval:
                maxval = mcc
                bestmetrics = mtry
        #store best model for the given fold and plot the metric vs mcc value
        rf = KNeighborsClassifier(n_neighbors=bestmetrics, weights='distance', 
                            algorithm='auto', leaf_size=100, p=2, metric='minkowski', 
                            metric_params=None, n_jobs=4)
        model_list.append(rf)
        testdat.append(maxval)
        totalmetrics.append(mcclist)

        #check models onto validation set, printing various metrics
        model= rf.fit(xtrain, ytrain)
        preds = model.predict(xvalid)
        results = metriccalc(preds, yvalid)
        validlist.append(results)
        print('for fold', split + 1, 'test set mcc of', maxval, 'valid set mcc of', results[12])
        
        iteration = iteration + 1
        
        #get x and y values for the final validation set

    toxvals = []
    fingerprints = []
    for row in tempset:
        toxvals.append(row[0])
        fingerprints.append(row[1:])
        
    #calculates consensus of models on each fingerprint in the final validation set
    consensuslist = []
    predictions = []
    for fp in fingerprints:
        consensus = -1
        fp = fp.reshape(1, -1)
        #appends each model's prediction to a list
        predictions = []
        predlist = []
        for model in model_list:
            preds = model.predict(fp)
            predictions.append(preds)
            predlist.append(preds)
        predictions.append(predlist)

        #finds number of 0s in the prediction list
        zercount = 0
        for num in predictions:
            if num == 0:
                zercount = zercount + 1
        #as there are 5 models, if there are less than 3 0s predicted, the consensus is 1
        if zercount < 3 :
            consensus = 1
        else:
            consensus = 0
        consensuslist.append(consensus)
    #calculate and print metrics
    results = metriccalc(consensuslist, toxvals)
    print('\nvalidation metrics of:')
    print('positives in data', results[0])
    print('negatives in data', results[1])
    print('net accuracy =', results[6])
    print('mcc =',results[12])
    sets += 1
    finalresults.append(results[12])

For dataset NR-AR-LBD_Avafp.csv
for fold 1 test set mcc of 0.686773437176115 valid set mcc of 0.44695292514227164
for fold 2 test set mcc of 0.5865557254410011 valid set mcc of 0.46046933104270676
for fold 3 test set mcc of 0.6546536707079772 valid set mcc of 0.3780500531015748
for fold 4 test set mcc of 0.7022915233992105 valid set mcc of 0.37609021559644396
for fold 5 test set mcc of 0.7876414620128274 valid set mcc of 0.4893451639269458

validation metrics of:
positives in data 47
negatives in data 49
net accuracy = 0.7604166666666666
mcc = 0.5224392925598775
For dataset SR-ATAD5_Morganfp.csv
for fold 1 test set mcc of 0.49348842518528757 valid set mcc of 0.12126781251816648
for fold 2 test set mcc of 0.5685575072471674 valid set mcc of 0.4690753506235187
for fold 3 test set mcc of 0.5038378351241242 valid set mcc of 0.3872983346207417
for fold 4 test set mcc of 0.36226673028138096 valid set mcc of 0.5262013605584339
for fold 5 test set mcc of 0.4947753344488257 valid set mcc of 0.3

for fold 4 test set mcc of 0.34345380351344046 valid set mcc of 0.3993086647358457
for fold 5 test set mcc of 0.47748110894004775 valid set mcc of 0.17383454240653054

validation metrics of:
positives in data 181
negatives in data 243
net accuracy = 0.6792452830188679
mcc = 0.3308801538385406
For dataset Hepatotoxicity_Avafp.csv
for fold 1 test set mcc of 0.5004204233921903 valid set mcc of 0.4376719512150403
for fold 2 test set mcc of 0.4198833387524375 valid set mcc of 0.3802335093329367
for fold 3 test set mcc of 0.5259842594645848 valid set mcc of 0.48306393688793137
for fold 4 test set mcc of 0.4978631002461275 valid set mcc of 0.44025810677056315
for fold 5 test set mcc of 0.4594632685398414 valid set mcc of 0.3814062929379806

validation metrics of:
positives in data 293
negatives in data 285
net accuracy = 0.7560553633217993
mcc = 0.514525791312545
For dataset Reproductive_Toxicity_Morganfp.csv
for fold 1 test set mcc of 0 valid set mcc of 0
for fold 2 test set mcc of 0 valid s

for fold 5 test set mcc of 0.7205766921228921 valid set mcc of 0.29707156852002076

validation metrics of:
positives in data 70
negatives in data 76
net accuracy = 0.7328767123287672
mcc = 0.4657214663500352
For dataset NR-AR_Morganfp.csv
for fold 1 test set mcc of 0.6572311106330365 valid set mcc of 0.6379052256590134
for fold 2 test set mcc of 0.5507419492112821 valid set mcc of 0.5015568278463085
for fold 3 test set mcc of 0.45613254692533467 valid set mcc of 0
for fold 4 test set mcc of 0.5393193716300061 valid set mcc of 0.2795426231258449
for fold 5 test set mcc of 0.6816381233958094 valid set mcc of 0

validation metrics of:
positives in data 0
negatives in data 0
net accuracy = 0
mcc = 0
For dataset Cardiotoxicity-5_MACCSfp.csv
for fold 1 test set mcc of 0.4382862782019066 valid set mcc of 0.267819448903787
for fold 2 test set mcc of 0.49692188043897767 valid set mcc of 0.3624420117874321
for fold 3 test set mcc of 0.45161290322580644 valid set mcc of 0.3444087584587639
for fol

for fold 4 test set mcc of 0.6519443563485797 valid set mcc of 0.6516235334190785
for fold 5 test set mcc of 0.5928553521950807 valid set mcc of 0.6142125693928948

validation metrics of:
positives in data 839
negatives in data 658
net accuracy = 0.8109552438209753
mcc = 0.614948522161922
For dataset NR-AR_TTorsionfp.csv
for fold 1 test set mcc of 0.6572311106330365 valid set mcc of 0.5069447770645309
for fold 2 test set mcc of 0.5618006477335896 valid set mcc of 0.600245047998781
for fold 3 test set mcc of 0.44702442937253495 valid set mcc of 0.5341637197842793
for fold 4 test set mcc of 0.6050751277598158 valid set mcc of 0.29417420270727607
for fold 5 test set mcc of 0.5819358307200044 valid set mcc of 0.4422689813358516

validation metrics of:
positives in data 61
negatives in data 64
net accuracy = 0.736
mcc = 0.5274949660495826
For dataset NR-ER_Avafp.csv
for fold 1 test set mcc of 0.3490711379181125 valid set mcc of 0.25918663677676024
for fold 2 test set mcc of 0.32936774553854

for fold 2 test set mcc of 0.5423006726618285 valid set mcc of 0.48738961726075464
for fold 3 test set mcc of 0.6595376088159248 valid set mcc of 0.5755368111731415
for fold 4 test set mcc of 0.6011919553182677 valid set mcc of 0.4852381077326207
for fold 5 test set mcc of 0.5866015554778286 valid set mcc of 0.45751808676292666

validation metrics of:
positives in data 164
negatives in data 106
net accuracy = 0.725925925925926
mcc = 0.42167948019344165
For dataset Respiratory_Toxicity_TTorsionfp.csv
for fold 1 test set mcc of 0.4782608695652174 valid set mcc of 0.37139320487678956
for fold 2 test set mcc of 0.5111777712211116 valid set mcc of 0.3498973717479126
for fold 3 test set mcc of 0.5374387982493612 valid set mcc of 0.46953134865677715
for fold 4 test set mcc of 0.6011919553182677 valid set mcc of 0.4525282220354418
for fold 5 test set mcc of 0.5741343728058275 valid set mcc of 0.33492956385764067

validation metrics of:
positives in data 164
negatives in data 106
net accuracy =

for fold 1 test set mcc of 0.4867902176053999 valid set mcc of 0.5491257635284318
for fold 2 test set mcc of 0.512491112209713 valid set mcc of 0.620903211826656
for fold 3 test set mcc of 0.5183745520649292 valid set mcc of 0.48283777191037763
for fold 4 test set mcc of 0.45729962601620733 valid set mcc of 0.3190454672827682
for fold 5 test set mcc of 0.5386248617532305 valid set mcc of 0.3918248643999888

validation metrics of:
positives in data 70
negatives in data 240
net accuracy = 0.8548387096774194
mcc = 0.5395818099391085
For dataset SR-MMP_Avafp.csv
for fold 1 test set mcc of 0.636797090063531 valid set mcc of 0.4059793854890171
for fold 2 test set mcc of 0.5541952714552937 valid set mcc of 0.577186375979705
for fold 3 test set mcc of 0.5088257224402446 valid set mcc of 0.509048435339345
for fold 4 test set mcc of 0.6157558485510554 valid set mcc of 0.43596901878529754
for fold 5 test set mcc of 0.6392610270322598 valid set mcc of 0.4742958421631853

validation metrics of:
pos

for fold 4 test set mcc of 0.40524896881636446 valid set mcc of 0.41518942736604736
for fold 5 test set mcc of 0.5184688572366651 valid set mcc of 0.16803478540247052

validation metrics of:
positives in data 73
negatives in data 79
net accuracy = 0.6447368421052632
mcc = 0.28755899927359346
For dataset Developmental_Toxicity_Morganfp.csv
for fold 1 test set mcc of 0 valid set mcc of 0
for fold 2 test set mcc of 0.16666666666666666 valid set mcc of 0.0
for fold 3 test set mcc of 0 valid set mcc of 0
for fold 4 test set mcc of 0 valid set mcc of 0
for fold 5 test set mcc of 0 valid set mcc of 0.16666666666666666

validation metrics of:
positives in data 8
negatives in data 6
net accuracy = 0.7857142857142857
mcc = 0.5594309277855158
For dataset Cardiotoxicity-30_Avafp.csv
for fold 1 test set mcc of 0.30952380952380953 valid set mcc of 0.1533557257490552
for fold 2 test set mcc of 0.327445652173913 valid set mcc of 0.3086066999241838
for fold 3 test set mcc of 0.31035490043061176 valid s

In [5]:
finalresults

[0.5224392925598775,
 0.33743567470752406,
 0.4646785977740792,
 0.4516591109168654,
 0.5723387958404564,
 0.5589979253939279,
 0.381897590901659,
 0.41156557439955455,
 0.35349951843867855,
 0.37841890351674645,
 0.3926767941066129,
 0.3863095765811034,
 0.4442276861859421,
 0.3308801538385406,
 0.514525791312545,
 0.05976143046671968,
 0.3402804183494084,
 -0.16666666666666666,
 0.4077671491175933,
 0.38940111458138327,
 0.45383877172149034,
 0.05976143046671968,
 0.30000599502710085,
 0.5305539523742169,
 0.5720642066959932,
 0.23529652646066698,
 0.3773936184383908,
 0.4657214663500352,
 0,
 0.3510045885850323,
 0.5770485509757954,
 0.2869749231780359,
 0.3591556191769535,
 0.6073664063908583,
 0.5984209925969702,
 0.05976143046671968,
 0.5939723189065484,
 0.5602117964757503,
 0.4605346858287855,
 0.42556070567620496,
 0.6134162532192503,
 0.614948522161922,
 0.5274949660495826,
 0.2903943999526783,
 0.3275731085241172,
 0.45992667090925055,
 0.3071149881268544,
 0.452070985301534

In [6]:
temp = pd.DataFrame(finalresults, columns=['MCC'])
temp2 = pd.DataFrame(namelist, columns=['Dataset'])
resultframe = pd.concat([temp2, temp], axis=1)
resultframe.to_csv('/Users/james/Documents/Honours/Results/Model_Outputs/KNN_Fingerprint_Results.csv')

In [14]:
resultframe

,Dataset,MCC
0,CarcinogenicityMorganfp.csv,0.300006
1,morganfp_SR-p53.csv,0.392793
2,morganfp_NR-AhR.csv,0.572339
3,morganfp_NR-ER-LBD.csv,0.453839
4,mutagenTTorsionfp.csv,0.514638
5,morganfp_NR-PPAR-gamma.csv,0.353500
6,Respiratory_ToxicityMorganfp.csv,0.421679
7,mutagenMACCSfp.csv,0.614949
8,Reproductive_ToxicityTTorsionfp.csv,0.000000
9,CarcinogenicityAvafp.csv,0.269629
